**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# The Postgres Operator by Zalando

see https://github.com/zalando/postgres-operator

In [1]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [3]:
git clone https://github.com/zalando/postgres-operator.git

Cloning into 'postgres-operator'...
remote: Enumerating objects: 22926, done.        
remote: Counting objects: 100% (50/50), done.        
remote: Compressing objects: 100% (40/40), done.        
remote: Total 22926 (delta 16), reused 23 (delta 8), pack-reused 22876        
Receiving objects: 100% (22926/22926), 8.58 MiB | 151.00 KiB/s, done.
Resolving deltas: 100% (16424/16424), done.


In [4]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [5]:
cd postgres-operator

In [ ]:
kubectl apply -f manifests/configmap.yaml  # configuration
kubectl apply -f manifests/operator-service-account-rbac.yaml  # identity and permissions
kubectl apply -f manifests/postgres-operator.yaml  # deployment
kubectl apply -f manifests/api-service.yaml  # operator API to be used by UI

configmap/postgres-operator created
serviceaccount/postgres-operator created
clusterrole.rbac.authorization.k8s.io/postgres-operator created
clusterrolebinding.rbac.authorization.k8s.io/postgres-operator created
clusterrole.rbac.authorization.k8s.io/postgres-pod created
deployment.apps/postgres-operator created
service/postgres-operator created


In [ ]:
helm install postgres-operator ./charts/postgres-operator

Error: INSTALLATION FAILED: rendered manifests contain a resource that already exists. Unable to continue with install: ServiceAccount "postgres-operator" in namespace "default" exists and cannot be imported into the current release: invalid ownership metadata; label validation error: missing key "app.kubernetes.io/managed-by": must be set to "Helm"; annotation validation error: missing key "meta.helm.sh/release-name": must be set to "postgres-operator"; annotation validation error: missing key "meta.helm.sh/release-namespace": must be set to "default"


: 1

In [11]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


No resources found in default namespace.


In [17]:
kubectl get pods -A

NAMESPACE     NAME                                       READY   STATUS    RESTARTS       AGE
default       acid-mypg-0                                1/1     Running   0              70s
default       acid-mypg-1                                1/1     Running   0              11s
default       acid-mypg-2                                1/1     Running   0              6s
default       postgres-operator-569b58b8c6-lh29h         1/1     Running   0              17m
default       postgres-operator-ui-58644cfcff-f6bd7      1/1     Running   0              9m5s
default       sleep-56bc8d768-w8dzz                      1/1     Running   0              151m
kube-system   calico-kube-controllers-58497c65d5-lptb7   1/1     Running   0              156m
kube-system   calico-node-424xg                          1/1     Running   0              156m
kube-system   coredns-78fcd69978-cvtqr                   1/1     Running   0              156m
kube-system   etcd-minikube                             

In [18]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS   AGE
pod/acid-mypg-0                             1/1     Running   0          98s
pod/acid-mypg-1                             1/1     Running   0          39s
pod/acid-mypg-2                             1/1     Running   0          34s
pod/postgres-operator-569b58b8c6-lh29h      1/1     Running   0          18m
pod/postgres-operator-ui-58644cfcff-f6bd7   1/1     Running   0          9m33s
pod/sleep-56bc8d768-w8dzz                   1/1     Running   0          151m

NAME                           TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/acid-mypg              ClusterIP   10.104.67.51    <none>        5432/TCP   98s
service/acid-mypg-config       ClusterIP   None            <none>        <none>     34s
service/acid-mypg-repl         ClusterIP   10.109.52.123   <none>        5432/TCP   98s
service/kubernetes             ClusterIP   10.96.0.1       <none>        443/TCP    157m
service/postgres

In [12]:
helm install postgres-operator-ui ./charts/postgres-operator-ui

NAME: postgres-operator-ui
LAST DEPLOYED: Thu Dec 16 11:07:29 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"


In [13]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"

NAME                                    READY   STATUS              RESTARTS   AGE
postgres-operator-ui-58644cfcff-f6bd7   0/1     ContainerCreating   0          8s


In [14]:
kubectl port-forward svc/postgres-operator-ui 8081:80

Forwarding from 127.0.0.1:8081 -> 8081


In [16]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS   AGE
pod/postgres-operator-569b58b8c6-lh29h      1/1     Running   0          13m
pod/postgres-operator-ui-58644cfcff-f6bd7   1/1     Running   0          4m28s
pod/sleep-56bc8d768-w8dzz                   1/1     Running   0          146m

NAME                           TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/kubernetes             ClusterIP   10.96.0.1       <none>        443/TCP    152m
service/postgres-operator      ClusterIP   10.96.176.129   <none>        8080/TCP   13m
service/postgres-operator-ui   ClusterIP   10.106.192.74   <none>        80/TCP     4m28s

NAME                                   READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/postgres-operator      1/1     1            1           13m
deployment.apps/postgres-operator-ui   1/1     1            1           4m28s
deployment.apps/sleep                  1/1     1            1           146m

NAME                    

# In a separate terminal

`kubectl port-forward --address $(hostname -i) svc/postgres-operator-ui 8081:80`

In [19]:
kubectl get secret

NAME                                                      TYPE                                  DATA   AGE
default-token-mfn9q                                       kubernetes.io/service-account-token   3      158m
postgres-operator-token-49lwq                             kubernetes.io/service-account-token   3      20m
postgres-operator-ui-token-bjvvq                          kubernetes.io/service-account-token   3      11m
postgres-pod-token-mtmxw                                  kubernetes.io/service-account-token   3      3m27s
postgres.acid-mypg.credentials.postgresql.acid.zalan.do   Opaque                                2      3m27s
sh.helm.release.v1.postgres-operator-ui.v1                helm.sh/release.v1                    1      11m
standby.acid-mypg.credentials.postgresql.acid.zalan.do    Opaque                                2      3m27s


In [20]:
kubectl get svc 

NAME                   TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
acid-mypg              ClusterIP   10.104.67.51    <none>        5432/TCP   4m7s
acid-mypg-config       ClusterIP   None            <none>        <none>     3m3s
acid-mypg-repl         ClusterIP   10.109.52.123   <none>        5432/TCP   4m7s
kubernetes             ClusterIP   10.96.0.1       <none>        443/TCP    159m
postgres-operator      ClusterIP   10.96.176.129   <none>        8080/TCP   20m
postgres-operator-ui   ClusterIP   10.106.192.74   <none>        80/TCP     12m


In [21]:
kubectl get service acid-mypg-repl -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-12-16T11:15:24Z"
  labels:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
    team: acid
  name: acid-mypg-repl
  namespace: default
  resourceVersion: "7276"
  uid: 101c0e53-a60a-4b0e-8147-de8259574dee
spec:
  clusterIP: 10.109.52.123
  clusterIPs:
  - 10.109.52.123
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: postgresql
    port: 5432
    protocol: TCP
    targetPort: 5432
  selector:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [22]:
export PGPASSWORD=$(kubectl get secret postgres.acid-mypg.credentials.postgresql.acid.zalan.do -o 'jsonpath={.data.password}' | base64 -d)
export PGSSLMODE=require

export PGHOST=acid-mypg-repl
#$(kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[0].ip}' )
echo $PGHOST

echo use
PGCOMMAND=$(echo PGPASSWORD=$PGPASSWORD PGSSLMODE=$PGSSLMODE  /usr/bin/psql  -h $PGHOST -U postgres)
echo $PGCOMMAND
echo to connect

acid-mypg-repl
use
PGPASSWORD=Pf7tcmfSercbnpddgmYU49JUOjIYyQbwoA0XwHIrC9d718tZaqYdh658dtyxq9YO PGSSLMODE=require /usr/bin/psql -h acid-mypg-repl -U postgres
to connect


In [24]:
kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[*].ip}'

In [25]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS   AGE
pod/acid-mypg-0                             1/1     Running   0          7m2s
pod/acid-mypg-1                             1/1     Running   0          6m3s
pod/acid-mypg-2                             1/1     Running   0          5m58s
pod/postgres-operator-569b58b8c6-lh29h      1/1     Running   0          23m
pod/postgres-operator-ui-58644cfcff-f6bd7   1/1     Running   0          14m
pod/sleep-56bc8d768-w8dzz                   1/1     Running   0          157m

NAME                           TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/acid-mypg              ClusterIP   10.104.67.51    <none>        5432/TCP   7m2s
service/acid-mypg-config       ClusterIP   None            <none>        <none>     5m58s
service/acid-mypg-repl         ClusterIP   10.109.52.123   <none>        5432/TCP   7m2s
service/kubernetes             ClusterIP   10.96.0.1       <none>        443/TCP    162m
service/po

In [ ]:
kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[0].ip}'

In [26]:
kubectl get service acid-mypg-repl -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-12-16T11:15:24Z"
  labels:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
    team: acid
  name: acid-mypg-repl
  namespace: default
  resourceVersion: "7276"
  uid: 101c0e53-a60a-4b0e-8147-de8259574dee
spec:
  clusterIP: 10.109.52.123
  clusterIPs:
  - 10.109.52.123
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: postgresql
    port: 5432
    protocol: TCP
    targetPort: 5432
  selector:
    application: spilo
    cluster-name: acid-mypg
    spilo-role: replica
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [27]:
echo $PGCOMMAND

PGPASSWORD=Pf7tcmfSercbnpddgmYU49JUOjIYyQbwoA0XwHIrC9d718tZaqYdh658dtyxq9YO PGSSLMODE=require /usr/bin/psql -h acid-mypg-repl -U postgres


In [28]:
kubectl exec -t  acid-mypg-0 -- /bin/bash -c "$PGCOMMAND -c 'select datname FROM pg_database'"

  datname  
-----------
 postgres
 template1
 template0
(3 rows)



In [34]:
kubectl get pods


NAME                                    READY   STATUS    RESTARTS   AGE
acid-mypg-0                             1/1     Running   0          5s
acid-mypg-1                             1/1     Running   0          9m8s
acid-mypg-2                             1/1     Running   0          9m3s
postgres-operator-569b58b8c6-lh29h      1/1     Running   0          26m
postgres-operator-ui-58644cfcff-f6bd7   1/1     Running   0          18m
sleep-56bc8d768-w8dzz                   1/1     Running   0          160m


In [36]:
kubectl logs acid-mypg-1 | grep  --color=always -E '$|leader'

2021-12-16 11:16:28,193 - bootstrapping - INFO - Figuring out my environment (Google? AWS? Openstack? Local?)
2021-12-16 11:16:30,221 - bootstrapping - INFO - Could not connect to 169.254.169.254, assuming local Docker setup
2021-12-16 11:16:30,224 - bootstrapping - INFO - No meta-data available for this provider
2021-12-16 11:16:30,225 - bootstrapping - INFO - Looks like your running local
2021-12-16 11:16:30,375 - bootstrapping - INFO - Configuring crontab
2021-12-16 11:16:30,376 - bootstrapping - INFO - Skipping creation of renice cron job due to lack of SYS_NICE capability
2021-12-16 11:16:30,376 - bootstrapping - INFO - Configuring pgqd
2021-12-16 11:16:30,377 - bootstrapping - INFO - Configuring wal-e
2021-12-16 11:16:30,377 - bootstrapping - INFO - Configuring bootstrap
2021-12-16 11:16:30,377 - bootstrapping - INFO - Configuring log
2021-12-16 11:16:30,377 - bootstrapping - INFO - Configuring standby-cluster
2021-12-16 11:16:30,377 - bootstrapping - INFO - Configuring pam-oauth

In [32]:
kubectl delete pod acid-mypg-0

pod "acid-mypg-0" deleted


In [37]:
kubectl get serviceaccount

NAME                   SECRETS   AGE
default                1         167m
postgres-operator      1         28m
postgres-operator-ui   1         20m
postgres-pod           1         12m


In [ ]:
kubectl get rolebinding

In [38]:
kubectl get rolebinding postgres-pod -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  creationTimestamp: "2021-12-16T11:15:24Z"
  name: postgres-pod
  namespace: default
  resourceVersion: "7267"
  uid: e766009f-9723-4fb6-bf1b-e471f3c7be42
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: postgres-pod
subjects:
- kind: ServiceAccount
  name: postgres-pod


In [39]:
kubectl get clusterrole postgres-pod -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"ClusterRole","metadata":{"annotations":{},"name":"postgres-pod"},"rules":[{"apiGroups":[""],"resources":["endpoints"],"verbs":["create","delete","deletecollection","get","list","patch","update","watch"]},{"apiGroups":[""],"resources":["pods"],"verbs":["get","list","patch","update","watch"]},{"apiGroups":[""],"resources":["services"],"verbs":["create"]}]}
  creationTimestamp: "2021-12-16T10:58:36Z"
  name: postgres-pod
  resourceVersion: "6476"
  uid: 6375e3eb-ef43-4ae3-be74-3e7a40bc3366
rules:
- apiGroups:
  - ""
  resources:
  - endpoints
  verbs:
  - create
  - delete
  - deletecollection
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - services
  verbs:
 

In [ ]:
kubectl delete clusterrole postgres-pod

In [ ]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  creationTimestamp: "2021-12-02T09:27:08Z"
  name: postgres-pod
  namespace: default
  resourceVersion: "47244"
  uid: 411eb7be-b67a-48d3-8df7-c30a6dc603ec
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: postgres-pod
subjects:
- kind: ServiceAccount
  name: postgres-pod
EOF

In [ ]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: postgres-pod
rules:
- apiGroups:
  - ""
  resources:
  - endpoints
  verbs:
  - create
  - delete
  - deletecollection
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
  - list
  - patch
  - update
  - watch
- apiGroups:
  - ""
  resources:
  - services
  verbs:
  - create
EOF

In [ ]:
kubectl get role postgres-pod -o yaml

In [40]:
kubectl auth can-i --as default:postgres-pod create pods

no


: 1

In [ ]:
kubectl get clusterrole postgres-operator-ui -o yaml

In [41]:
kubectl get crd

NAME                                                  CREATED AT
bgpconfigurations.crd.projectcalico.org               2021-12-16T08:39:42Z
bgppeers.crd.projectcalico.org                        2021-12-16T08:39:42Z
blockaffinities.crd.projectcalico.org                 2021-12-16T08:39:42Z
clusterinformations.crd.projectcalico.org             2021-12-16T08:39:43Z
felixconfigurations.crd.projectcalico.org             2021-12-16T08:39:43Z
globalnetworkpolicies.crd.projectcalico.org           2021-12-16T08:39:43Z
globalnetworksets.crd.projectcalico.org               2021-12-16T08:39:43Z
hostendpoints.crd.projectcalico.org                   2021-12-16T08:39:43Z
ipamblocks.crd.projectcalico.org                      2021-12-16T08:39:43Z
ipamconfigs.crd.projectcalico.org                     2021-12-16T08:39:43Z
ipamhandles.crd.projectcalico.org                     2021-12-16T08:39:43Z
ippools.crd.projectcalico.org                         2021-12-16T08:39:43Z
kubecontrollersconfigurations.crd.p

In [42]:
kubectl get crd postgresqls.acid.zalan.do -o yaml

apiVersion: apiextensions.k8s.io/v1
kind: CustomResourceDefinition
metadata:
  creationTimestamp: "2021-12-16T10:58:46Z"
  generation: 1
  name: postgresqls.acid.zalan.do
  resourceVersion: "6511"
  uid: 928a2035-3a29-4fb5-a97c-3885b5faa545
spec:
  conversion:
    strategy: None
  group: acid.zalan.do
  names:
    categories:
    - all
    kind: postgresql
    listKind: postgresqlList
    plural: postgresqls
    shortNames:
    - pg
    singular: postgresql
  scope: Namespaced
  versions:
  - additionalPrinterColumns:
    - description: Team responsible for Postgres cluster
      jsonPath: .spec.teamId
      name: Team
      type: string
    - description: PostgreSQL version
      jsonPath: .spec.postgresql.version
      name: Version
      type: string
    - description: Number of Pods per Postgres cluster
      jsonPath: .spec.numberOfInstances
      name: Pods
      type: integer
    - description: Size of the bound volume
      jsonPath: .spec.volume.size
      name: Volume
      t

In [ ]:
kubectl get postgresqls.acid.zalan.do 

In [ ]:
kubectl get postgresqls.acid.zalan.do acid-mypg -o yaml

In [ ]:
kubectl create -f - <<EOF
apiVersion: acid.zalan.do/v1
kind: postgresql
metadata:
  name: acid-mypg-yaml
  namespace: default
spec:
  allowedSourceRanges: null
  numberOfInstances: 3
  postgresql:
    version: "14"
  resources:
    limits:
      cpu: 500m
      memory: 500Mi
    requests:
      cpu: 100m
      memory: 100Mi
  teamId: acid
  volume:
    size: 10Gi
EOF

In [ ]:
kubectl get all